<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/365_EFIA_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Agent Workflow & Orchestration

### From Raw Feedback to Executive Intelligence

This code block defines the **operational backbone** of the Employee Feedback Intelligence Agent.

Rather than relying on implicit execution or opaque chains, the agent is built as a **clear sequence of nodes**, each with a specific responsibility, explicit inputs, and predictable outputs. This makes the system easy to reason about, debug, govern, and scale.

In short:
this is where the agent *earns trust*.

---

## Why a Node-Based Design Matters

Many AI agents blur logic, data, and interpretation into a single flow.
This agent does the opposite.

Each node:

* Has a single, well-defined purpose
* Reads from state
* Writes back to state
* Handles its own errors explicitly

This creates a workflow that is:

* Transparent
* Inspectable
* Resilient
* Easy to extend

Leadership doesn’t just see *results* — they can understand *how those results were produced*.

---

## Goal & Planning: Making Intent Explicit

### `goal_node`

The agent begins by clearly defining **why it exists**.

The goal node captures:

* The business objective
* The analytical focus areas
* The expected outputs

This may seem simple — but it’s a powerful design choice.

It ensures the agent always operates with **declared intent**, not hidden assumptions.

---

### `planning_node`

Once the goal is defined, the agent builds a **step-by-step execution plan**.

This plan:

* Is rule-based (no LLM needed)
* Makes dependencies explicit
* Defines what each step produces

If someone asks:

> “What happens first, and why?”

The answer is literally documented in the plan.

---

## Core Processing Nodes

Each of the following nodes performs one logical stage of reasoning.

### Data Loading Node

Loads validated feedback data from JSON files and stores it in state.

If data is missing or invalid, the agent stops early — preventing downstream contamination.

---

### Aggregation Node

Groups feedback by:

* Department
* Category (Issue vs Idea)
* Recurring themes

This is where individual comments become **organizational patterns**.

---

### Sentiment Analysis Node

Adds emotional context to feedback using transparent, rule-based logic.

Sentiment is captured as *supporting signal*, not as a decision-maker.

---

### Prioritization Node

Applies Pareto-style reasoning to identify:

* The issues driving the most friction
* The ideas with the highest potential impact

Every priority includes a **human-readable rationale**, ensuring explainability.

---

## LLM Usage: Carefully Scoped and Controlled

### Summarization Node

This node is where LLMs are used — and *only* where they add value.

The LLM:

* Summarizes departments
* Explains key themes
* Produces an executive summary

Critically:

* All inputs are structured
* All conclusions are pre-determined by rules
* The LLM explains — it does not decide

Summarization can also be **disabled entirely via configuration**, reinforcing governance and control.

---

## Visualization as a First-Class Output

### Visualization Node

This node generates all charts:

* Issues by department
* Ideas by department
* Sentiment distribution
* Top priority issues
* Top priority ideas

Each visualization is:

* Deterministic
* Saved to disk
* Referenced explicitly in the report

This ensures insights are:

* Shareable
* Comparable over time
* Ready for leadership review

---

## Final Output: A Complete Intelligence Report

### Report Generation Node

The final node assembles everything into a **single markdown intelligence report**:

* Executive summary
* Key statistics
* Sentiment overview
* Top issues and ideas
* Visual references
* Department summaries
* Error log (if any)

The report is automatically saved and versioned, creating a **permanent audit trail**.

---

## Error Handling & Resilience

Across all nodes:

* Errors are collected, not hidden
* Partial failures don’t crash the entire system
* Failures are surfaced clearly in the final report

This makes the agent safe to run in real operational environments.

---

## Architectural Takeaway

This node-based orchestration layer is what transforms a collection of analytics into a **true intelligence agent**.

It ensures:

* Intent is explicit
* Logic is inspectable
* AI is constrained
* Outputs are executive-ready
* Trust is preserved at every step

In practical terms:

> **The agent doesn’t just produce insight — it shows its work.**



# Nodes for Employee Feedback Intelligence Agent

In [ ]:
"""Nodes for Employee Feedback Intelligence Agent

Orchestrates the workflow for analyzing employee feedback.
"""

from typing import Dict, Any
from config import EmployeeFeedbackIntelligenceState, EmployeeFeedbackIntelligenceConfig

# Import utilities
from agents.employee_feedback_intelligence.utilities.data_loading import (
    load_all_feedback_files
)
from agents.employee_feedback_intelligence.utilities.aggregation import (
    aggregate_by_department,
    aggregate_by_category,
    calculate_feedback_summary,
    detect_themes
)
from agents.employee_feedback_intelligence.utilities.sentiment import (
    analyze_all_feedback_sentiment,
    calculate_sentiment_summary
)
from agents.employee_feedback_intelligence.utilities.prioritization import (
    prioritize_issues,
    prioritize_ideas,
    calculate_prioritization_summary
)
from agents.employee_feedback_intelligence.utilities.summarization import (
    generate_department_summary,
    generate_theme_summary,
    generate_executive_summary
)
from agents.employee_feedback_intelligence.utilities.visualization import (
    create_issues_by_department_chart,
    create_ideas_by_department_chart,
    create_sentiment_by_department_chart,
    create_top_issues_chart,
    create_top_ideas_chart
)
from toolshed.reporting import save_report


def goal_node(state: EmployeeFeedbackIntelligenceState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for feedback analysis.

    This is a simple rule-based goal definition.
    """
    goal = {
        "objective": "Transform employee feedback into actionable organizational intelligence",
        "focus_areas": [
            "aggregation_and_trend_detection",
            "sentiment_analysis",
            "prioritization_80_20",
            "llm_summarization",
            "visual_intelligence"
        ],
        "outputs": [
            "feedback_summary",
            "prioritized_issues",
            "prioritized_ideas",
            "executive_summary",
            "visualizations",
            "intelligence_report"
        ]
    }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: EmployeeFeedbackIntelligenceState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    plan = [
        {
            "step": 1,
            "name": "data_loading",
            "description": "Load feedback data from JSON files",
            "dependencies": [],
            "outputs": ["raw_feedback"]
        },
        {
            "step": 2,
            "name": "aggregation",
            "description": "Aggregate feedback by department, category, and detect themes",
            "dependencies": ["data_loading"],
            "outputs": ["feedback_by_department", "feedback_by_category", "feedback_summary", "themes"]
        },
        {
            "step": 3,
            "name": "sentiment_analysis",
            "description": "Analyze sentiment for all feedback entries",
            "dependencies": ["data_loading"],
            "outputs": ["sentiment_analysis", "sentiment_summary"]
        },
        {
            "step": 4,
            "name": "prioritization",
            "description": "Prioritize issues and ideas using Pareto (80/20) principle",
            "dependencies": ["aggregation", "sentiment_analysis"],
            "outputs": ["prioritized_issues", "prioritized_ideas", "prioritization_summary"]
        },
        {
            "step": 5,
            "name": "summarization",
            "description": "Generate LLM-based summaries for departments, themes, and executive summary",
            "dependencies": ["aggregation", "sentiment_analysis", "prioritization"],
            "outputs": ["department_summaries", "theme_summaries", "executive_summary"]
        },
        {
            "step": 6,
            "name": "visualization",
            "description": "Generate bar charts and visualizations",
            "dependencies": ["aggregation", "sentiment_analysis", "prioritization"],
            "outputs": ["visualization_paths"]
        },
        {
            "step": 7,
            "name": "report_generation",
            "description": "Generate final intelligence report",
            "dependencies": ["summarization", "visualization"],
            "outputs": ["feedback_intelligence_report", "report_file_path"]
        }
    ]

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }


def data_loading_node(
    state: EmployeeFeedbackIntelligenceState,
    config: EmployeeFeedbackIntelligenceConfig
) -> Dict[str, Any]:
    """
    Data Loading Node: Load feedback data from JSON files.
    """
    errors = state.get("errors", [])
    data_dir = state.get("data_dir") or config.data_dir

    try:
        raw_feedback = load_all_feedback_files(data_dir, config.feedback_files)

        return {
            "raw_feedback": raw_feedback,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: {str(e)}"]
        }


def aggregation_node(
    state: EmployeeFeedbackIntelligenceState,
    config: EmployeeFeedbackIntelligenceConfig
) -> Dict[str, Any]:
    """
    Aggregation Node: Group feedback by department, category, and detect themes.
    """
    errors = state.get("errors", [])
    raw_feedback = state.get("raw_feedback", [])

    if not raw_feedback:
        return {
            "errors": errors + ["aggregation_node: raw_feedback is required"]
        }

    try:
        feedback_by_department = aggregate_by_department(raw_feedback)
        feedback_by_category = aggregate_by_category(raw_feedback)
        feedback_summary = calculate_feedback_summary(raw_feedback)
        themes = detect_themes(raw_feedback, min_frequency=config.min_theme_frequency)

        return {
            "feedback_by_department": feedback_by_department,
            "feedback_by_category": feedback_by_category,
            "feedback_summary": feedback_summary,
            "themes": themes,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"aggregation_node: {str(e)}"]
        }


def sentiment_analysis_node(
    state: EmployeeFeedbackIntelligenceState,
    config: EmployeeFeedbackIntelligenceConfig
) -> Dict[str, Any]:
    """
    Sentiment Analysis Node: Analyze sentiment for all feedback.
    """
    errors = state.get("errors", [])
    raw_feedback = state.get("raw_feedback", [])

    if not raw_feedback:
        return {
            "errors": errors + ["sentiment_analysis_node: raw_feedback is required"]
        }

    try:
        sentiment_analysis = analyze_all_feedback_sentiment(
            raw_feedback,
            config.sentiment_keywords
        )
        sentiment_summary = calculate_sentiment_summary(raw_feedback, sentiment_analysis)

        return {
            "sentiment_analysis": sentiment_analysis,
            "sentiment_summary": sentiment_summary,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"sentiment_analysis_node: {str(e)}"]
        }


def prioritization_node(
    state: EmployeeFeedbackIntelligenceState,
    config: EmployeeFeedbackIntelligenceConfig
) -> Dict[str, Any]:
    """
    Prioritization Node: Prioritize issues and ideas using Pareto (80/20) principle.
    """
    errors = state.get("errors", [])
    raw_feedback = state.get("raw_feedback", [])
    themes = state.get("themes", [])
    sentiment_analysis = state.get("sentiment_analysis", [])

    if not raw_feedback:
        return {
            "errors": errors + ["prioritization_node: raw_feedback is required"]
        }

    try:
        prioritized_issues = prioritize_issues(
            raw_feedback,
            themes,
            sentiment_analysis,
            top_n=config.top_n_issues
        )

        prioritized_ideas = prioritize_ideas(
            raw_feedback,
            themes,
            sentiment_analysis,
            top_n=config.top_n_ideas
        )

        prioritization_summary = calculate_prioritization_summary(
            prioritized_issues,
            prioritized_ideas
        )

        return {
            "prioritized_issues": prioritized_issues,
            "prioritized_ideas": prioritized_ideas,
            "prioritization_summary": prioritization_summary,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"prioritization_node: {str(e)}"]
        }


def summarization_node(
    state: EmployeeFeedbackIntelligenceState,
    config: EmployeeFeedbackIntelligenceConfig
) -> Dict[str, Any]:
    """
    Summarization Node: Generate LLM-based summaries.
    """
    errors = state.get("errors", [])

    if not config.enable_llm_summarization:
        # Skip summarization if disabled
        return {
            "department_summaries": {},
            "theme_summaries": {},
            "executive_summary": "LLM summarization disabled.",
            "errors": errors
        }

    raw_feedback = state.get("raw_feedback", [])
    feedback_by_department = state.get("feedback_by_department", {})
    themes = state.get("themes", [])
    sentiment_summary = state.get("sentiment_summary", {})
    feedback_summary = state.get("feedback_summary", {})
    prioritized_issues = state.get("prioritized_issues", [])
    prioritized_ideas = state.get("prioritized_ideas", [])

    try:
        # Generate department summaries
        department_summaries = {}
        for department, entries in feedback_by_department.items():
            try:
                summary = generate_department_summary(
                    department,
                    entries,
                    sentiment_summary,
                    llm_model=config.llm_model,
                    temperature=config.temperature,
                    max_tokens=config.max_summary_tokens
                )
                department_summaries[department] = summary
            except Exception as e:
                errors.append(f"Error generating summary for {department}: {str(e)}")
                department_summaries[department] = f"Summary generation failed for {department}."

        # Generate theme summaries (top N themes)
        theme_summaries = {}
        top_themes = themes[:config.summarize_top_n_themes]
        for theme in top_themes:
            try:
                # Get feedback entries for this theme
                theme_feedback_ids = set(theme.get("feedback_ids", []))
                theme_entries = [e for e in raw_feedback if e.get("submission_id") in theme_feedback_ids]

                summary = generate_theme_summary(
                    theme,
                    theme_entries,
                    llm_model=config.llm_model,
                    temperature=config.temperature,
                    max_tokens=config.max_summary_tokens
                )
                theme_summaries[theme.get("theme_id")] = summary
            except Exception as e:
                errors.append(f"Error generating summary for theme {theme.get('theme_id')}: {str(e)}")
                theme_summaries[theme.get("theme_id")] = f"Summary generation failed."

        # Generate executive summary
        try:
            executive_summary = generate_executive_summary(
                feedback_summary,
                prioritized_issues,
                prioritized_ideas,
                sentiment_summary,
                llm_model=config.llm_model,
                temperature=config.temperature,
                max_tokens=config.max_summary_tokens
            )
        except Exception as e:
            errors.append(f"Error generating executive summary: {str(e)}")
            executive_summary = "Executive summary generation failed."

        return {
            "department_summaries": department_summaries,
            "theme_summaries": theme_summaries,
            "executive_summary": executive_summary,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"summarization_node: {str(e)}"]
        }


def visualization_node(
    state: EmployeeFeedbackIntelligenceState,
    config: EmployeeFeedbackIntelligenceConfig
) -> Dict[str, Any]:
    """
    Visualization Node: Generate bar charts and visualizations.
    """
    errors = state.get("errors", [])
    feedback_summary = state.get("feedback_summary", {})
    sentiment_summary = state.get("sentiment_summary", {})
    prioritized_issues = state.get("prioritized_issues", [])
    prioritized_ideas = state.get("prioritized_ideas", [])

    try:
        visualization_paths = {}

        # Create charts directory
        charts_dir = config.charts_dir

        # Issues by department
        try:
            path = create_issues_by_department_chart(
                feedback_summary,
                charts_dir,
                width=config.chart_width,
                height=config.chart_height,
                dpi=config.chart_dpi
            )
            visualization_paths["issues_by_department"] = path
        except Exception as e:
            errors.append(f"Error creating issues by department chart: {str(e)}")

        # Ideas by department
        try:
            path = create_ideas_by_department_chart(
                feedback_summary,
                charts_dir,
                width=config.chart_width,
                height=config.chart_height,
                dpi=config.chart_dpi
            )
            visualization_paths["ideas_by_department"] = path
        except Exception as e:
            errors.append(f"Error creating ideas by department chart: {str(e)}")

        # Sentiment by department
        try:
            path = create_sentiment_by_department_chart(
                sentiment_summary,
                charts_dir,
                width=config.chart_width,
                height=config.chart_height,
                dpi=config.chart_dpi
            )
            visualization_paths["sentiment_by_department"] = path
        except Exception as e:
            errors.append(f"Error creating sentiment by department chart: {str(e)}")

        # Top issues
        try:
            path = create_top_issues_chart(
                prioritized_issues,
                charts_dir,
                top_n=min(10, len(prioritized_issues)),
                width=config.chart_width,
                height=config.chart_height,
                dpi=config.chart_dpi
            )
            visualization_paths["top_issues"] = path
        except Exception as e:
            errors.append(f"Error creating top issues chart: {str(e)}")

        # Top ideas
        try:
            path = create_top_ideas_chart(
                prioritized_ideas,
                charts_dir,
                top_n=min(10, len(prioritized_ideas)),
                width=config.chart_width,
                height=config.chart_height,
                dpi=config.chart_dpi
            )
            visualization_paths["top_ideas"] = path
        except Exception as e:
            errors.append(f"Error creating top ideas chart: {str(e)}")

        return {
            "visualization_paths": visualization_paths,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"visualization_node: {str(e)}"]
        }


def report_generation_node(
    state: EmployeeFeedbackIntelligenceState,
    config: EmployeeFeedbackIntelligenceConfig
) -> Dict[str, Any]:
    """
    Report Generation Node: Generate final intelligence report.
    """
    errors = state.get("errors", [])

    try:
        report = _generate_feedback_intelligence_report(state)

        # Save report
        report_file_path = save_report(
            report,
            "employee_feedback_intelligence",
            reports_dir=config.reports_dir,
            prefix="feedback_intelligence_report"
        )

        return {
            "feedback_intelligence_report": report,
            "report_file_path": report_file_path,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"report_generation_node: {str(e)}"]
        }


def _generate_feedback_intelligence_report(state: EmployeeFeedbackIntelligenceState) -> str:
    """Generate markdown report from state."""
    report = "# Employee Feedback Intelligence Report\n\n"

    # Executive Summary
    executive_summary = state.get("executive_summary", "No executive summary available.")
    report += f"## Executive Summary\n\n{executive_summary}\n\n---\n\n"

    # Overall Statistics
    feedback_summary = state.get("feedback_summary", {})
    report += "## Overall Statistics\n\n"
    report += f"- **Total Feedback:** {feedback_summary.get('total_feedback', 0)}\n"
    report += f"- **Total Issues:** {feedback_summary.get('total_issues', 0)}\n"
    report += f"- **Total Ideas:** {feedback_summary.get('total_ideas', 0)}\n"
    report += f"- **Departments:** {', '.join(feedback_summary.get('departments', []))}\n\n"

    # Sentiment Summary
    sentiment_summary = state.get("sentiment_summary", {})
    report += "## Sentiment Analysis\n\n"
    report += f"- **Overall Sentiment:** {sentiment_summary.get('overall_sentiment', 'neutral')}\n"
    report += f"- **Positive:** {sentiment_summary.get('positive_count', 0)}\n"
    report += f"- **Neutral:** {sentiment_summary.get('neutral_count', 0)}\n"
    report += f"- **Negative:** {sentiment_summary.get('negative_count', 0)}\n"
    report += f"- **Average Intensity:** {sentiment_summary.get('average_intensity', 0.0)}\n\n"

    # Top Priority Issues
    prioritized_issues = state.get("prioritized_issues", [])
    if prioritized_issues:
        report += "## Top Priority Issues\n\n"
        for i, issue in enumerate(prioritized_issues[:10], 1):
            report += f"### {i}. Priority Score: {issue.get('priority_score', 0):.1f}\n\n"
            report += f"**Department:** {issue.get('department', 'Unknown')}\n\n"
            report += f"**Feedback:** {issue.get('feedback_text', '')}\n\n"
            report += f"**Rationale:** {issue.get('rationale', '')}\n\n"

    # Top Priority Ideas
    prioritized_ideas = state.get("prioritized_ideas", [])
    if prioritized_ideas:
        report += "## Top Priority Ideas\n\n"
        for i, idea in enumerate(prioritized_ideas[:10], 1):
            report += f"### {i}. Priority Score: {idea.get('priority_score', 0):.1f}\n\n"
            report += f"**Department:** {idea.get('department', 'Unknown')}\n\n"
            report += f"**Feedback:** {idea.get('feedback_text', '')}\n\n"
            report += f"**Rationale:** {idea.get('rationale', '')}\n\n"

    # Visualizations
    visualization_paths = state.get("visualization_paths", {})
    if visualization_paths:
        report += "## Visualizations\n\n"
        for chart_name, chart_path in visualization_paths.items():
            report += f"- **{chart_name.replace('_', ' ').title()}:** `{chart_path}`\n"
        report += "\n"

    # Department Summaries
    department_summaries = state.get("department_summaries", {})
    if department_summaries:
        report += "## Department Summaries\n\n"
        for department, summary in department_summaries.items():
            report += f"### {department}\n\n{summary}\n\n"

    # Errors (if any)
    errors = state.get("errors", [])
    if errors:
        report += "## Errors\n\n"
        for error in errors:
            report += f"- {error}\n"
        report += "\n"

    return report

